### Import packages to graph results

In [9]:
from glob import glob
import pandas as pd
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'iframe'
import os
import import_ipynb
from shared_notebook import create_dataframe

importing Jupyter notebook from shared_notebook.ipynb
Stored 'colorPallet' (list)
Stored 'bgColor' (str)
Stored 'baseFigureWidth' (int)
Stored 'baseFigureHeight' (int)


### Set default color palete and size for making plots

In [10]:
%store -r colorPallet
%store -r bgColor
%store -r baseFigureWidth
%store -r baseFigureHeight

### glob all the log files into a list, then create a dataframe with all json data

In [11]:
def create_dataframe(logs):
    logs = glob(logs)
    df_list = []
    for log in logs:
        with open(log) as f:
            obj = json.load(f)['saaf']
        # cpuInfo is messing up the dataframe... remove for now
        obj.pop('cpuInfo')
        df_list.append(pd.DataFrame([obj]))
    return pd.concat(df_list, ignore_index=True)

In [12]:
dna_df = create_dataframe("../logs/openfaas/kops-multi-tenant-spot-1x-8vcpu-16gb/sebs/*dna*")

In [13]:
bfs_df = create_dataframe("../logs/openfaas/kops-multi-tenant-spot-1x-8vcpu-16gb/sebs/*bfs*")

In [14]:
mst_df = create_dataframe("../logs/openfaas/kops-multi-tenant-spot-1x-8vcpu-16gb/sebs/*mst*")

In [15]:
pagerank_df = create_dataframe("../logs/openfaas/kops-multi-tenant-spot-1x-8vcpu-16gb/sebs/*pagerank*")

# Make bar plots to profile sebs function data

In [16]:
fig = make_subplots(specs = [[{"secondary_y": False}]])

# Add figure title
fig.update_layout(
    barmode='stack',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.47
    ),
    margin=dict(
        t=0,
        b=1,
        l=1,
        r=1,
        autoexpand=True
    ),
    font=dict(
        size=16
    ),
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    plot_bgcolor=bgColor
)

profileData = pd.DataFrame()
profileData['payload'] = ["dna_visualization", "graph_bfs", "graph_mst", "graph_pagerank"]
# Metrics for profiling
metrics = ['SoftIrq', 'Kernel', 'IOWait', 'User', 'Idle']
profileData['cpuTotalTime'] = [0, 0, 0, 0]
for metric in metrics:
    dnaMean = dna_df[f'cpu{metric}Delta'].mean()
    bfsMean = bfs_df[f'cpu{metric}Delta'].mean()
    mstMean = mst_df[f'cpu{metric}Delta'].mean()
    pagerankMean = pagerank_df[f'cpu{metric}Delta'].mean()
    profileData[metric] = [dnaMean, bfsMean, mstMean, pagerankMean]
    profileData['cpuTotalTime'] = [
        dnaMean + profileData['cpuTotalTime'][0],
        bfsMean + profileData['cpuTotalTime'][1],
        mstMean + profileData['cpuTotalTime'][2],
        pagerankMean + profileData['cpuTotalTime'][3]
    ]

#profileData['runtime'] = [round(normal_df['runtime'].mean() / 1000), round(tumor_df['runtime'].mean() / 1000)]
#profileData['runtimeText'] = [f"Runtime:<br>{value} s" for value in profileData['runtime']]
for i, metric in enumerate(metrics):
    profileData[metric] = [
        profileData[metric][0] * 100 / profileData['cpuTotalTime'][0],
        profileData[metric][1] * 100 / profileData['cpuTotalTime'][1],
        profileData[metric][2] * 100 / profileData['cpuTotalTime'][2],
        profileData[metric][3] * 100 / profileData['cpuTotalTime'][3]
    ]
    text = [f'{round(val, 2)}%' for val in profileData[metric]]
    fig.add_trace(go.Bar(x = profileData["payload"],
                         y = profileData[metric],
                         name = f"CPU {metric}", marker_color=colorPallet[i],
                         text = text),
                  secondary_y=False)

# Set x-axis title
fig.update_xaxes(title_text="Functions")

# Set y-axes titles
fig.update_yaxes(title_text="CPU Time Percentage", secondary_y=False)
# fig.update_yaxes(title_text="Function Runtime (ms)", secondary_y=True)

# py.iplot(fig)

fig.show()
# Save graph
# Save graph
if not os.path.exists("graphs"):
    os.mkdir("graphs")
fig.write_image("graphs/sebs_profile_bars.png", width=baseFigureWidth, height=baseFigureHeight)